# Импорт модулей

In [ ]:
import pandas as pd
import numpy as np
import time
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
import collections
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import category_encoders as ce

from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import f_classif # anova
from sklearn.model_selection import train_test_split  # специальный инструмент для разбивки
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Задаем параметры

In [ ]:
DATA_DIR = '/kaggle/input/sf-booking/'
TARGET_NAME = 'reviewer_score'
RANDOM_SEED = 42

# Загружаем данные

In [ ]:
train_data = pd.read_csv(DATA_DIR + '/hotels_train.csv') # датасет для обучения
test_data = pd.read_csv(DATA_DIR + 'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR + '/submission.csv') # сабмишн

# Чтение

In [ ]:
train_data.head(3)

In [ ]:
test_data.head(3)

In [ ]:
sample_submission.head(3)

Найдем дебликаты

In [ ]:
mask = train_data.duplicated(subset=train_data.columns) # маска для фильтрации
data_duplicates = train_data[mask] # фильтруем наш датасет
print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')

Удалим дубликаты

In [ ]:
train_data.drop_duplicates(inplace=True)

# Объединяем данные

In [ ]:
train_data['sample'] = 1 # данные для обучения
train_data['id'] = train_data.index
test_data['sample'] = 0 # данные для теста
test_data['reviewer_score'] = 0 # заполняем нулями тестовые данные для предсказания
test_data['id'] = test_data.index # задаем id в тестовых данных
data = train_data.copy()
data = pd.concat([train_data,test_data],axis=0,ignore_index=True)

data.info()

# Очистка данных

Исследуем данные на наличие пропусков

In [ ]:
cols_null_percent = data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
cols_with_null

Количество пропусков в признаках lat и lng значительное (37%). Кроме того, данные признаки представляют собой географические координаты. Логически уровень отеля скорее зависит от района, в котором он расположен, чем от страны/города (где в одном городе отели могу иметь совершенно разную оценку), поэтому заполнение пропусков средним, медианным, модальным значением или нулем будет не объективно при таком большом их количестве. Данные признаки удалим.

In [ ]:
data.drop(['lat', 'lng'], axis=1, inplace=True)

# Проектирование признаков

Извлечем количество дней между датой проверки и датой очистки в целочисленном формате и удалим старый признак.

In [ ]:
regex = '\d+'
data['days_since_review_int'] = data['days_since_review'].str.findall(regex).str.get(0).astype('int')
data.drop(['days_since_review'], axis=1, inplace=True)
data.info()

Извлечем из адреса страну и город отеля и запишем в новый признак. Добавим данные о количестве населения и площади города в отдельные признаки. Удалим исходный признак.

In [ ]:
data['hotel_city'] = data['hotel_address'].apply(lambda x: 'London' if x.endswith('United Kingdom') else x.split()[-2])

сity_population = {
    'Paris':  2148327, 'London': 8908081, 'Milan': 1366180, 
    'Vienna': 1911191, 'Barcelona': 1636732, 'Amsterdam': 860124
}
сity_area = {
    'Paris': 105, 'London': 1706, 'Milan': 181, 
    'Vienna': 414, 'Barcelona': 101, 'Amsterdam': 219
}

data['сity_population'] = data['hotel_city'].map(сity_population)
data['сity_area'] = data['hotel_city'].map(сity_area)

data.drop(['hotel_address'], axis=1, inplace=True)

data.info()

Проанализируем позитивные и негативные отзывы с помощью SentimentIntensityAnalyzer. Результаты занесем в отдельные признаки. Исходные признаки удалим.

In [ ]:
sent_analyzer = SentimentIntensityAnalyzer()
start_time = time.time()
pos = data["positive_review"].apply(lambda x: abs(sent_analyzer.polarity_scores(x)["compound"]))
neg = data["negative_review"].apply(lambda x: -abs(sent_analyzer.polarity_scores(x)["compound"]))

data["sentiment_score"] = pos + neg
data["polarity_pos"] = pos
data["polarity_neg"] = neg

time_model = time.time() - start_time
print(f"Execution time: {int(time_model)} seconds")

data.drop(['positive_review', 'negative_review'], axis=1, inplace=True)

data.info()

Выделим признаки месяца и года из даты отзыва, предварительно приведя признак к формату datetime. Удалим первоначальный признак даты.

In [ ]:
data['review_date'] = pd.to_datetime(data['review_date'], format='%m/%d/%Y')
data['review_day'] = data['review_date'].dt.day.astype(int)
data['review_month'] = data['review_date'].dt.month.astype(int)
data['review_year'] = data['review_date'].dt.year.astype(int)
data['review_weekday'] = data['review_date'].dt.dayofweek.astype(int)
data.drop(['review_date'], axis=1, inplace=True)

data.info()

Преобразуем колонку 'tags' таким образом, чтобы остались только 10 наиболее популярных тэгов в значениях.

In [ ]:
data['tags'] = data['tags'].apply(lambda x: x.lower())
data['tags_list'] = data['tags'].apply(lambda x: x[3:-3].split(" ', ' "))

# Получаем топ-10 тегов
top_tags = list(data['tags'].explode().value_counts().index)[:10]

# Реализуем вариант OneHotEncoder для топ-10 тегов
for t in top_tags:
    data[t] = data['tags'].apply(lambda x: 1 if t in x else 0)

data = data.drop(['tags'], axis=1)

data.info()

## Кодировка признаков

Произведем кодировку признака hotel_city методом однократного кодирования (так как количество значений менее 10). Удалим исходный признак.

In [ ]:
encoder = ce.OneHotEncoder(cols=['hotel_city'])
type_bin = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin], axis=1)

data.drop(['hotel_city'], axis=1, inplace=True)
data.info()

Произведем кодировку признака 'reviewer_nationality' методом двоичного кодирования (так как количество значений более 200). Удалим исходный признак.

In [ ]:
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality'])
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

data.drop(['reviewer_nationality'], axis=1, inplace=True)
data.info()

Произведем кодировку признака hotel_name методом двоичного кодирования (так как количество значений более 1400). Удалим исходный признак.

In [ ]:
bin_encoder = ce.BinaryEncoder(cols=['hotel_name'])
type_bin = bin_encoder.fit_transform(data['hotel_name'])
data = pd.concat([data, type_bin], axis=1)

data.drop(['hotel_name'], axis=1, inplace=True)
data.info()

Удалим оставшиеся текстовые признаки

In [ ]:
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)
data.info()

# Разделим данные

In [ ]:
train_data = data[data['sample'] == 1]
test_data = data[data['sample'] == 0]
train_data.index = train_data['id']
test_data.index = test_data['id']

train_data.drop(['sample','id'],axis=1,inplace=True)
test_data.drop(['sample','id','reviewer_score'],axis=1,inplace=True)

train_data.info()

# Разделим признаки на непрерывные и категориальные и оценим их влияние.

In [ ]:
# непрерывные признаки
num_cols = ['average_score', 'sentiment_score', 'polarity_pos', 'polarity_neg', 
            'total_number_of_reviews', 'review_total_negative_word_counts', 
            'review_total_positive_word_counts', 
            'total_number_of_reviews_reviewer_has_given',
            'additional_number_of_scoring', 'days_since_review_int', 
            'сity_population', 'сity_area']

# категориальные признаки
cat_cols = ['review_day', 'review_month', 'review_year', 'review_weekday']   #'tags'

Оценим на влияние категориальных признаков

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  

X = train_data.drop(['reviewer_score'], axis = 1)  
y = train_data['reviewer_score'] 

y=y.astype('int')

plt.rcParams['figure.figsize'] = (15,10)
imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Оценим на влияние числовых признаков

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Удалим колонки с низким рейтингом

In [ ]:
train_data.drop(['review_year', 'days_since_review_int'],axis=1, inplace=True)
test_data.drop(['review_year', 'days_since_review_int'],axis=1, inplace=True)
test_data.info()

Оценим корреляцию оставшихся признаков

In [ ]:
plt.rcParams['figure.figsize'] = (30,30)
sns.heatmap(train_data.drop('reviewer_score',axis=1).corr(), annot=True, cmap='coolwarm')

Index([additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'reviewer_score',
       'country_population', 'sentiment_score', 'polarity_pos', 'polarity_neg',
       'review_day', 'review_month', 'review_year', 'review_weekday',
       'hotel_country_1', 'hotel_country_2', 'hotel_country_3',
       'hotel_country_4', 'hotel_country_5', 'hotel_country_6', 'hotel_city_1',
       'hotel_city_2', 'hotel_city_3', 'hotel_city_4', 'hotel_city_5',
       'hotel_city_6', 'reviewer_nationality_0', 'reviewer_nationality_1',
       'reviewer_nationality_2', 'reviewer_nationality_3',
       'reviewer_nationality_4', 'reviewer_nationality_5',
       'reviewer_nationality_6', 'reviewer_nationality_7', 'hotel_name_0',
       'hotel_name_1', 'hotel_name_2', 'hotel_name_3', 'hotel_name_4',
       'hotel_name_5', 'hotel_name_6', 'hotel_name_7', 'hotel_name_8',
       'hotel_name_9', 'hotel_name_10', 'tags'],

Удалим признаки с мультиколлинеарностью

In [ ]:
train_data.drop(['additional_number_of_scoring', 'сity_population'], axis=1, inplace=True)
test_data.drop(['additional_number_of_scoring', 'сity_population'], axis=1, inplace=True)

Проверяем корреляцию

In [ ]:
plt.rcParams['figure.figsize'] = (30,30)
sns.heatmap(train_data.drop('reviewer_score',axis=1).corr(), annot=True, cmap='coolwarm')

Доудалим признаки с мультиколлинеарностью

In [ ]:
train_data.drop(['sentiment_score', 'сity_area'], axis=1, inplace=True)
test_data.drop(['sentiment_score', 'сity_area'], axis=1, inplace=True)

In [ ]:
plt.rcParams['figure.figsize'] = (30,30)
sns.heatmap(train_data.drop('reviewer_score',axis=1).corr(), annot=True, cmap='coolwarm')

# Обучаем модель

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = train_data.drop(['reviewer_score'], axis = 1)  
y = train_data['reviewer_score'] 


# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# проверяем размерности получившихся датасетов
print(test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape)

In [ ]:
# Создаём модель
model = RandomForestRegressor(n_estimators=500, verbose=1, random_state=RANDOM_SEED, n_jobs=-1)  
      
# Обучаем модель на тестовом наборе данных  
model.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = model.predict(X_test)


# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

# MAPE: 0.12985432963200785

Выведем самые важные признаки для модели

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Выполним предсказание тестовых данных

In [ ]:
test_pred = model.predict(test_data)
# сохраним в файл
sample_submission['reviewer_score'] = test_pred
sample_submission.to_csv('predict_mape.csv', index = False)